In [1]:
# append the parent folder of ark to sys.path
import sys
sys.path.append(r"E:\Users\Claris_Gu\PycharmProjects\python_project\Cell Segmentation\astir")

In [5]:
import anndata
from astir.data import from_csv_yaml # ignoring warnings since importing from local installation of astir
import yaml
import os

## Load ICI-mean-intensity data

In [6]:
sep = '\t'
usecols = ['Cell_ID', 'CD45', 'CD4', 'CD68', 'CD20', 
            'CD8a', 'SOX10', 'CD40', 'CD3', 'S100', 'CD14', 
            'CD31']
ICI_dir = 'ICI'

In [7]:
# Read in yaml file for illustration purposes
with open(os.path.join(ICI_dir, 'ICI_markers.yml'), 'r') as file:
  cell_types = yaml.load(file)

  print(cell_types['cell_types'])

{'Cytotoxic T cell': ['CD3', 'CD45', 'CD8a'], 'Helper T cell': ['CD3', 'CD45', 'CD4'], 'B cell': ['CD20'], 'Melenoma cell': ['SOX10', 'S100'], 'Vascular differentiation': ['CD31'], 'Macrophage': ['CD14', 'CD68']}


In [8]:
ast = from_csv_yaml(os.path.join(ICI_dir, "ICI_meanIntensity.tsv"), os.path.join(ICI_dir, "ICI_markers.tsv"), sep=sep, usecols=usecols)
ast

TypeError: from_csv_yaml() got an unexpected keyword argument 'sep'